## 1. 패키지 로드

In [1]:
!pwd

/content


In [3]:
!unzip "/content/drive/MyDrive/인공지능시스템/plant_dataset.zip" -d "./dataset/"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./dataset/plant_dataset/Apple___healthy/image (1445).JPG  
  inflating: ./dataset/__MACOSX/plant_dataset/Apple___healthy/._image (1445).JPG  
  inflating: ./dataset/plant_dataset/Apple___healthy/image (262).JPG  
  inflating: ./dataset/__MACOSX/plant_dataset/Apple___healthy/._image (262).JPG  
  inflating: ./dataset/plant_dataset/Apple___healthy/image (1015).JPG  
  inflating: ./dataset/__MACOSX/plant_dataset/Apple___healthy/._image (1015).JPG  
  inflating: ./dataset/plant_dataset/Apple___healthy/image (798).JPG  
  inflating: ./dataset/__MACOSX/plant_dataset/Apple___healthy/._image (798).JPG  
  inflating: ./dataset/plant_dataset/Apple___healthy/image (510).JPG  
  inflating: ./dataset/__MACOSX/plant_dataset/Apple___healthy/._image (510).JPG  
  inflating: ./dataset/plant_dataset/Apple___healthy/image (1337).JPG  
  inflating: ./dataset/__MACOSX/plant_dataset/Apple___healthy/._image (1337).JPG  
  inflating: ./dataset/plant_dataset/App

In [32]:
import os
import shutil
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision import models
from torchvision.datasets import ImageFolder
import time
import copy

## 2. 데이터 분할
- 데이터 분할을 위한 디렉토리 생성

In [25]:
original_dataset_dir = '/content/dataset/plant_dataset'
classes_list = os.listdir(original_dataset_dir)

In [26]:
classes_list

['Apple___Cedar_apple_rust',
 'Apple___Apple_scab',
 'Potato___Early_blight',
 'Tomato___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn___Common_rust',
 'Pepper,_bell___healthy',
 'Cherry___Powdery_mildew',
 'Tomato___Tomato_mosaic_virus',
 'Grape___Black_rot',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Cherry___healthy',
 'Corn___Northern_Leaf_Blight',
 'Tomato___Leaf_Mold',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Tomato___Late_blight',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Corn___healthy',
 'Tomato___Target_Spot',
 'Grape___Esca_(Black_Measles)',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Apple___healthy',
 'Apple___Black_rot',
 'Tomato___Septoria_leaf_spot',
 'Grape___healthy']

In [27]:
base_dir = './splitted'
try:
    os.mkdir(base_dir)
except:
    pass

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')
try:
    os.mkdir(train_dir)
    os.mkdir(validation_dir)
    os.mkdir(test_dir)
except:
    pass

for cls in classes_list:
    try:
        os.mkdir(os.path.join(train_dir, cls))
        os.mkdir(os.path.join(validation_dir, cls))
        os.mkdir(os.path.join(test_dir, cls))
    except:
        pass

In [28]:
%cd /content/dataset/plant_dataset
!ls -al
!rm .DS_Store

/content/dataset/plant_dataset
total 1524
drwx------ 36 root root   4096 Nov 25 10:39  .
drwxr-xr-x  4 root root   4096 Nov 25 10:26  ..
drwxr-xr-x  2 root root  20480 Nov 24 13:13  Apple___Apple_scab
drwxr-xr-x  2 root root  20480 Nov 24 13:13  Apple___Black_rot
drwxr-xr-x  2 root root  12288 Nov 24 13:13  Apple___Cedar_apple_rust
drwxr-xr-x  2 root root  69632 Nov 24 13:13  Apple___healthy
drwxr-xr-x  2 root root  36864 Nov 24 13:13  Cherry___healthy
drwxr-xr-x  2 root root  36864 Nov 24 13:13  Cherry___Powdery_mildew
drwxr-xr-x  2 root root  20480 Nov 24 13:13 'Corn___Cercospora_leaf_spot Gray_leaf_spot'
drwxr-xr-x  2 root root  45056 Nov 24 13:13  Corn___Common_rust
drwxr-xr-x  2 root root  36864 Nov 24 13:13  Corn___healthy
drwxr-xr-x  2 root root  36864 Nov 24 13:13  Corn___Northern_Leaf_Blight
drwxr-xr-x  2 root root  40960 Nov 24 13:13  Grape___Black_rot
drwxr-xr-x  2 root root  45056 Nov 24 13:13 'Grape___Esca_(Black_Measles)'
drwxr-xr-x  2 root root  20480 Nov 24 13:13  Grape

In [29]:
for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)

    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)

    train_fnames = fnames[:train_size]
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)

    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)

    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]
    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Train size( Apple___Cedar_apple_rust ):  165
Validation size( Apple___Cedar_apple_rust ):  55
Test size( Apple___Cedar_apple_rust ):  55
Train size( Apple___Apple_scab ):  378
Validation size( Apple___Apple_scab ):  126
Test size( Apple___Apple_scab ):  126
Train size( Potato___Early_blight ):  600
Validation size( Potato___Early_blight ):  200
Test size( Potato___Early_blight ):  200
Train size( Tomato___healthy ):  954
Validation size( Tomato___healthy ):  318
Test size( Tomato___healthy ):  318
Train size( Pepper,_bell___Bacterial_spot ):  598
Validation size( Pepper,_bell___Bacterial_spot ):  199
Test size( Pepper,_bell___Bacterial_spot ):  199
Train size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  307
Validation size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  102
Test size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  102
Train size( Corn___Common_rust ):  715
Validation size( Corn___Common_rust ):  238
Test size( Corn___Common_rust ):  238
Train size( Pepper,_bell___he

## 3. Baseline 모델

In [30]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 512
EPOCH = 20

In [34]:
train_transform = T.Compose([
    T.Resize((64,64)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ToTensor(),
])

val_transform = T.Compose([
    T.Resize((64,64)),
    T.ToTensor(),
])

train_data = ImageFolder(root="./splitted/train", transform=train_transform)
val_data = ImageFolder(root="./splitted/val", transform=val_transform)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

#### 원본코드

In [31]:
# class Net(nn.Module):

#     def __init__(self):

#         super(Net, self).__init__()

#         self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
#         self.pool = nn.MaxPool2d(2,2)
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         self.conv3 = nn.Conv2d(64, 64, 3, padding=1)

#         self.fc1 = nn.Linear(4096, 512)
#         self.fc2 = nn.Linear(512, 33)

#     def forward(self, x):

#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.pool(x)
#         x = F.dropout(x, p=0.25, training=self.training)

#         x = self.conv2(x)
#         x = F.relu(x)
#         x = self.pool(x)
#         x = F.dropout(x, p=0.25, training=self.training)

#         x = self.conv3(x)
#         x = F.relu(x)
#         x = self.pool(x)
#         x = F.dropout(x, p=0.25, training=self.training)

#         x = x.view(-1, 4096)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.fc2(x)

#         return F.log_softmax(x, dim=1)

In [42]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv_layers = nn.ModuleList([
        nn.Conv2d(3, 32, 3, padding=1),
        nn.Conv2d(32, 64, 3, padding=1),
        nn.Conv2d(64, 64, 3, padding=1),
    ])

    self.fc_layers = nn.ModuleList([
        nn.Linear(4096, 512),
        nn.Linear(512, 33)
    ])

    self.pool = nn.MaxPool2d(2, 2)

  def forward(self, x):
    for conv in self.conv_layers:
      x = conv(x)
      x = F.relu(x)
      x = self.pool(x)
      x = F.dropout(x, p=0.25, training=self.training)

    x = x.view(-1, 4096)
    # 첫 번째 전결합 레이어 처리
    x = self.fc_layers[0](x)
    x = F.relu(x)
    x = F.dropout(x, p=0.5, training=self.training)

    # 두 번째 전결합 레이어 처리
    x = self.fc_layers[1](x)

    return F.log_softmax(x, dim=1)

### 4. Optimizer, Scheduler, Loss 함수 만들기

In [43]:
model_base = Net().to(DEVICE)
# Build optimizer
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

# Build scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCH * len(train_loader))

# Build loss function
loss_fn = nn.CrossEntropyLoss()

### 4. Train Loop 만들기

In [44]:
def train(model, train_loader, optimzer):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)

    #Forward
    outputs = model(data)
    loss = loss_fn(outputs, target)

    #Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #Update lr
    scheduler.step()

### 5. Evaluation Loop 만들기

In [47]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += loss_fn(output,target)


            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

### 6. Main Loop 만들기

In [48]:
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    model.load_state_dict(best_model_wts)
    return model


base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)  	 #(16)
torch.save(base,'baseline.pt')

-------------- epoch 1 ----------------
train Loss: 0.0028, Accuracy: 57.02%
val Loss: 0.0029, Accuracy: 57.22%
Completed in 1m 28s
-------------- epoch 2 ----------------
train Loss: 0.0021, Accuracy: 68.00%
val Loss: 0.0021, Accuracy: 67.43%
Completed in 1m 30s
-------------- epoch 3 ----------------
train Loss: 0.0018, Accuracy: 71.28%
val Loss: 0.0018, Accuracy: 71.65%
Completed in 1m 30s
-------------- epoch 4 ----------------
train Loss: 0.0015, Accuracy: 75.91%
val Loss: 0.0016, Accuracy: 75.40%
Completed in 1m 29s
-------------- epoch 5 ----------------
train Loss: 0.0014, Accuracy: 78.08%
val Loss: 0.0014, Accuracy: 78.02%
Completed in 1m 28s
-------------- epoch 6 ----------------
train Loss: 0.0011, Accuracy: 82.25%
val Loss: 0.0012, Accuracy: 81.99%
Completed in 1m 29s
-------------- epoch 7 ----------------
train Loss: 0.0011, Accuracy: 81.98%
val Loss: 0.0011, Accuracy: 82.25%
Completed in 1m 30s
-------------- epoch 8 ----------------
train Loss: 0.0011, Accuracy: 82.05%

### 7. 모델 평가하기

In [49]:
transform_base = transforms.Compose([transforms.Resize([64,64]),transforms.ToTensor()])
test_base = ImageFolder(root='./splitted/test',transform=transform_base)
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [50]:
baseline=torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test acc:  ', test_accuracy)

baseline test acc:   87.08223807735637


### Transfer Learning - Pretrained Model 활용하기